In [2]:
import os
import face_recognition
from PIL import Image
import cv2
import math
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset,random_split,Subset
import torch.optim as optim

In [22]:
# Move model to GPU if available
device = torch.device("cuda:0")
print(device)

cuda:0


In [5]:
def extract_faces_from_frames(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(input_folder, filename)
            image = face_recognition.load_image_file(image_path)

            # Find all face locations in the image
            face_locations = face_recognition.face_locations(image)

            # Extract and save faces
            for i, (top, right, bottom, left) in enumerate(face_locations):
                # only extract faces greater than 75x75 pixels --> helps to ignore wrong faces from background objects
                if (bottom - top > 75) and (right - left > 75):
                    face_roi = image[top:bottom, left:right]
                    face_path = os.path.join(
                        output_folder, f"{os.path.splitext(filename)[0]}_face_{i}.jpg"
                    )
                    face_image = Image.fromarray(face_roi)
                    face_image.save(face_path)
                    print(f"Saved face {i} from {filename}")

input_folder = "train/real1"
output_folder = "train/real2"


extract_faces_from_frames(input_folder, output_folder)

Saved face 0 from 01__exit_phone_room_2.jpg
Saved face 0 from 01__exit_phone_room_3.jpg
Saved face 0 from 01__exit_phone_room_4.jpg
Saved face 0 from 01__exit_phone_room_5.jpg
Saved face 0 from 01__exit_phone_room_6.jpg
Saved face 0 from 01__exit_phone_room_7.jpg
Saved face 0 from 01__exit_phone_room_8.jpg
Saved face 0 from 01__exit_phone_room_9.jpg
Saved face 0 from 01__hugging_happy_0.jpg
Saved face 0 from 01__hugging_happy_1.jpg
Saved face 0 from 01__hugging_happy_2.jpg
Saved face 0 from 01__hugging_happy_3.jpg
Saved face 0 from 01__hugging_happy_4.jpg
Saved face 0 from 01__hugging_happy_5.jpg
Saved face 0 from 01__hugging_happy_8.jpg
Saved face 0 from 01__hugging_happy_9.jpg
Saved face 0 from 01__kitchen_pan_0.jpg
Saved face 0 from 01__kitchen_pan_1.jpg
Saved face 0 from 01__kitchen_pan_2.jpg
Saved face 0 from 01__kitchen_pan_3.jpg
Saved face 0 from 01__kitchen_pan_4.jpg
Saved face 0 from 01__kitchen_pan_5.jpg
Saved face 0 from 01__kitchen_pan_6.jpg
Saved face 0 from 01__kitchen_pa

In [17]:

class DeepfakeDetectionDataset(Dataset):
    def __init__(self, root_dir, transform=None):

        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels = []

        for label, subdir in enumerate(['real2', 'fake2']):
            dir_path = os.path.join(self.root_dir, subdir)
            for file_name in os.listdir(dir_path):
                if file_name.endswith(('.jpg', '.jpeg', '.png')):  # Check file extension if necessary
                    img_path = os.path.join(dir_path, file_name)
                    image = face_recognition.load_image_file(img_path)
                    face_locations = face_recognition.face_locations(image)

                    # Only add the image if a suitable face is found
                    for top, right, bottom, left in face_locations:
                        if (bottom - top > 75) and (right - left > 75):
                            self.data.append(img_path)
                            self.labels.append(label)
                            break  # Found a suitable face, stop checking

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        img_path = self.data[idx]
        image = face_recognition.load_image_file(img_path)

        face_locations = face_recognition.face_locations(image)
        top, right, bottom, left = face_locations[0]  # Use the first suitable face
        face_image = image[top:bottom, left:right]
        face_image = Image.fromarray(face_image)
        
        if self.transform:
            face_image = self.transform(face_image)

        label = self.labels[idx]
        return face_image, label

In [18]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [19]:
dataset = DeepfakeDetectionDataset(root_dir='train',transform=transform)

In [33]:
# Split sizes
total_size = len(dataset)
# print(total_size)
# train_loader = DataLoader(dataset, batch_size=16, shuffle=True)
train_size = int(0.7 * total_size)
test_size = total_size - train_size
# val_size = total_size - train_size - test_size  # Remainder for validation
print(test_size)
print(train_size)
# print(val_size)
total_samples = len(dataset)
indices = torch.randperm(total_samples)[:16]  # Randomly shuffle and select the first 100 indices

small_train_dataset = Subset(dataset, indices)
small_train_loader = DataLoader(small_train_dataset, batch_size=16, shuffle=True)

# Split the dataset
# train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# # DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

17
39


In [21]:
def inspect_dataloader(dataloader):
    # Get the first batch
    images, labels = next(iter(dataloader))
    
    # Print the shapes and labels of the batch
    print(f"Batch shape: {images.shape}")
    print(f"Labels: {labels}")

print("Inspecting Training DataLoader:")
inspect_dataloader(train_loader)

# print("\nInspecting Validation DataLoader:")
# inspect_dataloader(val_loader)

# print("\nInspecting Testing DataLoader:")
# inspect_dataloader(test_loader)

Inspecting Training DataLoader:
Batch shape: torch.Size([16, 3, 224, 224])
Labels: tensor([1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1])


In [34]:
# Load a pre-trained ResNet50 model
model = models.resnet50(weights="ResNet50_Weights.IMAGENET1K_V1")

# Modify the final layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Output for 2 classes: fake and real

# model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [35]:
def train_model(model, criterion, optimizer, dataloader, num_epochs=5):
    model.train()  # Set model to training mode
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_preds = 0
        
        for inputs, labels in dataloader:
            print(f"Input batch shape: {inputs.shape}")
            print(f"Labels batch shape: {labels.shape}")
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = correct_preds.double() / len(dataloader.dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')
        
    print('Training complete')
    return model

# Example usage:
# Adjust `dataloader` to your training DataLoader



In [36]:
train_model(model, criterion, optimizer, train_loader, num_epochs=5)

Input batch shape: torch.Size([16, 3, 224, 224])
Labels batch shape: torch.Size([16])
Input batch shape: torch.Size([16, 3, 224, 224])
Labels batch shape: torch.Size([16])
Input batch shape: torch.Size([7, 3, 224, 224])
Labels batch shape: torch.Size([7])
Epoch 1/5, Loss: 0.6520, Acc: 0.7179
Input batch shape: torch.Size([16, 3, 224, 224])
Labels batch shape: torch.Size([16])
Input batch shape: torch.Size([16, 3, 224, 224])
Labels batch shape: torch.Size([16])
Input batch shape: torch.Size([7, 3, 224, 224])
Labels batch shape: torch.Size([7])
Epoch 2/5, Loss: 0.5479, Acc: 0.8205
Input batch shape: torch.Size([16, 3, 224, 224])
Labels batch shape: torch.Size([16])
Input batch shape: torch.Size([16, 3, 224, 224])
Labels batch shape: torch.Size([16])
Input batch shape: torch.Size([7, 3, 224, 224])
Labels batch shape: torch.Size([7])
Epoch 3/5, Loss: 0.7515, Acc: 0.7949
Input batch shape: torch.Size([16, 3, 224, 224])
Labels batch shape: torch.Size([16])
Input batch shape: torch.Size([16, 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [39]:
def evaluate_model(model, dataloader):
    model.eval()  # Set model to evaluate mode
    total = 0
    correct = 0
    
    with torch.no_grad():  # Inference without gradient calculation
        for inputs, labels in dataloader:
            outputs = model(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Evaluate the model
evaluate_model(model, test_loader)

Test Accuracy: 58.82%
